### Dataset Link:
```
https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-product-file-definitions
```

# Importing all the required libraries



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from wordcloud import WordCloud
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import joblib
import shap
warnings.filterwarnings('ignore')

# Creating folders to save outputs and models
os.makedirs('outputs', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Setting the memory usage less space
plt.rcParams['figure.max_open_warning'] = 50
plt.rcParams['figure.dpi'] = 100

# from google.colab import drive
# drive.mount('/content/drive')

# Data loading in a Otimized way



In [3]:
def load_data(file_path):
    """
    Load the FDA drug product data with memory optimization
    """
    print(f"Loading data from {file_path}...")


    try:
        # Load only necessary columns
        df = pd.read_excel(
            file_path,
            usecols=['PROPRIETARYNAME', 'NONPROPRIETARYNAME', 'ROUTENAME',
                     'DOSAGEFORMNAME', 'SUBSTANCENAME', 'PHARM_CLASSES',
                     'LABELERNAME', 'MARKETINGCATEGORYNAME',
                     'STARTMARKETINGDATE']
        )
        print(f"Data loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
        print(f"Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Data Cleaning and Preprocessing

In [4]:
def clean_and_preprocess_data(df):
    """
    Clean and preprocess the FDA drug product data
    """
    print("Starting data cleaning and preprocessing...")

    # Ploting original distribution of MARKETINGCATEGORYNAME
    original_distribution = df['MARKETINGCATEGORYNAME'].value_counts()
    plt.figure(figsize=(14, 8))
    ax = original_distribution.plot(kind='bar', color=sns.color_palette("viridis", len(original_distribution)))
    plt.title('Original Marketing Category Distribution')
    plt.xlabel('Marketing Category')
    plt.ylabel('Count')
    plt.xticks(rotation=90)

    # Adding count labels on top of bars
    for i, v in enumerate(original_distribution.values):
        ax.text(i, v + 5, str(v), ha='center')
    plt.tight_layout()
    plt.savefig('outputs/original_category_distribution.jpg')
    plt.close()

    # Group MARKETINGCATEGORYNAME into consolidated labels
    marketing_category_mapping = {
        'OTC MONOGRAPH DRUG': 'OTC',
        'OTC MONOGRAPH FINAL': 'OTC',
        'OTC MONOGRAPH NOT FINAL': 'OTC',
        'UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE': 'UNAPPROVED',
        'UNAPPROVED MEDICAL GAS': 'UNAPPROVED',
        'UNAPPROVED DRUG OTHER': 'UNAPPROVED',
        'UNAPPROVED HOMEOPATHIC': 'UNAPPROVED',
        'NDA': 'NDA',
        'NDA AUTHORIZED GENERIC': 'NDA',
        'ANDA': 'ANDA',
        'BLA': 'BLA'
    }

    # Creating a new column with grouped categories
    df['GROUPED_MARKETING_CATEGORY'] = df['MARKETINGCATEGORYNAME'].map(
        marketing_category_mapping, na_action='ignore')

    # Drop rows with EMERGENCY USE AUTHORIZATION and Null values in grouped target
    df = df[df['MARKETINGCATEGORYNAME'] != 'EMERGENCY USE AUTHORIZATION']
    df = df.dropna(subset=['GROUPED_MARKETING_CATEGORY'])

    # Fill textual nulls with 'no text' to avoid the null and affect the performance
    text_columns = [
        'PROPRIETARYNAME', 'NONPROPRIETARYNAME', 'ROUTENAME',
        'DOSAGEFORMNAME', 'SUBSTANCENAME', 'PHARM_CLASSES',
        'LABELERNAME'
    ]
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna('no text')

    # Convert STARTMARKETINGDATE to datetime if present to use in EDA
    if 'STARTMARKETINGDATE' in df.columns:
        df['STARTMARKETINGDATE'] = pd.to_datetime(df['STARTMARKETINGDATE'], errors='coerce')
        df['MARKETING_YEAR'] = df['STARTMARKETINGDATE'].dt.year

    # Ploting grouped distribution
    grouped_distribution = df['GROUPED_MARKETING_CATEGORY'].value_counts()
    plt.figure(figsize=(10, 6))
    ax = grouped_distribution.plot(kind='bar', color=sns.color_palette("viridis", len(grouped_distribution)))
    plt.title('Grouped Marketing Category Distribution')
    plt.xlabel('Marketing Category')
    plt.ylabel('Count')

    # To add count labels on top of bars
    for i, v in enumerate(grouped_distribution.values):
        ax.text(i, v + 5, str(v), ha='center')
    plt.tight_layout()
    plt.savefig('outputs/grouped_category_distribution.jpg')
    plt.close()

    # Creating a missing value heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
    plt.title('Missing Value Heatmap')
    plt.tight_layout()
    plt.savefig('outputs/missing_value_heatmap.jpg')
    plt.close()

    # Plot Year wise listing distribution
    if 'MARKETING_YEAR' in df.columns:
        plt.figure(figsize=(14, 6))
        year_counts = df['MARKETING_YEAR'].value_counts().sort_index()
        year_counts.plot(kind='line', marker='o')
        plt.title('Year-wise Drug Listing Distribution')
        plt.xlabel('Year')
        plt.ylabel('Number of Listings')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('outputs/year_wise_distribution.jpg')
        plt.close()

    df = df.reset_index(drop=True)
    print(f"Data cleaned and preprocessed. Remaining rows: {df.shape[0]}")
    return df

# Feature Engineering

In [5]:
def engineer_features(df):
    """
    Engineer features for modeling with memory optimization
    """
    print("Starting feature engineering...")

    df['GROUPED_MARKETING_CATEGORY'] = df['GROUPED_MARKETING_CATEGORY'].astype('category')

    # Text length and word count features
    for col in ['PROPRIETARYNAME', 'DOSAGEFORMNAME', 'SUBSTANCENAME', 'NONPROPRIETARYNAME']:
        if col in df.columns:
            df[f'{col}_LENGTH'] = df[col].str.len()
            df[f'{col}_WORD_COUNT'] = df[col].str.split().str.len()

    # Activing Ingredient Count
    if 'SUBSTANCENAME' in df.columns:
        df['ACTIVE_INGREDIENT_COUNT'] = df['SUBSTANCENAME'].str.count(';') + 1
        df.loc[df['SUBSTANCENAME'] == 'no text', 'ACTIVE_INGREDIENT_COUNT'] = 0

    # Pharmaceutical Class Count (imputed with -1 due to overfitting and deviating the features from actual learning, earlier tried with 0)
    if 'PHARM_CLASSES' in df.columns:
        df['PHARM_CLASS_COUNT'] = df['PHARM_CLASSES'].str.count(';') + 1
        df.loc[df['PHARM_CLASSES'] == 'no text', 'PHARM_CLASS_COUNT'] = -1

        key_classes = ['analgesic', 'antibiotic', 'antiviral', 'cardiovascular', 'central nervous system',
                       'anti-inflammatory', 'hormone', 'vaccine', 'immunologic']
        for cls in key_classes:
            df[f'IS_{cls.upper().replace(" ", "_")}'] = df['PHARM_CLASSES'].str.contains(cls, case=False, na=False).astype(np.int8)

    # TF-IDF for SUBSTANCENAME
    if 'SUBSTANCENAME' in df.columns:
        print("Generating TF-IDF features for SUBSTANCENAME...")
        tfidf = TfidfVectorizer(max_features=50, stop_words='english')
        substance_tfidf = tfidf.fit_transform(df['SUBSTANCENAME'].fillna('no text'))
        tfidf_feature_names = [f'SUBSTANCE_TFIDF_{i}' for i in range(substance_tfidf.shape[1])]
        substance_tfidf_df = pd.DataFrame(substance_tfidf.toarray(), columns=tfidf_feature_names)
        df = pd.concat([df, substance_tfidf_df], axis=1)
        print("Top TF-IDF terms:")
        feature_names = tfidf.get_feature_names_out()
        for i, feature_name in enumerate(feature_names[:10]):
            print(f"        {feature_name}")

    # Route-specific features
    if 'ROUTENAME' in df.columns:
        route_mapping = {
            'ORAL': ['ORAL', 'BY MOUTH', 'ENTERAL'],
            'TOPICAL': ['TOPICAL', 'CUTANEOUS', 'TRANSDERMAL', 'SKIN'],
            'INJECTION': ['INJECTION', 'INTRAVENOUS', 'INTRAMUSCULAR', 'SUBCUTANEOUS'],
            'OPHTHALMIC': ['OPHTHALMIC', 'EYE'],
            'NASAL': ['NASAL', 'INTRANASAL'],
            'RECTAL': ['RECTAL'],
            'VAGINAL': ['VAGINAL'],
            'INHALATION': ['INHALATION', 'RESPIRATORY (INHALATION)']
        }
        for route_category, route_terms in route_mapping.items():
            pattern = '|'.join(route_terms)
            df[f'IS_ROUTE_{route_category}'] = df['ROUTENAME'].str.contains(pattern, case=False, na=False).astype(np.int8)

    # Memory optimization
    int_columns = [col for col in df.columns if '_COUNT' in col or 'IS_' in col]
    for col in int_columns:
        df[col] = df[col].fillna(0).astype(np.int8)

    float_columns = [col for col in df.columns if '_LENGTH' in col or 'TFIDF' in col]
    for col in float_columns:
        df[col] = df[col].astype(np.float16)

    gc.collect()
    print("Feature engineering completed")
    print(f"Memory usage after engineering: {df.memory_usage().sum() / 1024**2:.2f} MB")
    return df

# Data Prepration for Model training and testing

In [6]:
def prepare_data_for_modeling(df):
    """
    Prepare the data for modeling with memory optimization
    """
    print("Preparing data for modeling...")

    # 5.1 Select important features
    # First, get all available columns by type
    text_cols = ['PROPRIETARYNAME', 'DOSAGEFORMNAME', 'ROUTENAME', 'LABELERNAME']
    length_cols = [col for col in df.columns if '_LENGTH' in col]
    count_cols = [col for col in df.columns if '_COUNT' in col]
    binary_cols = [col for col in df.columns if 'IS_' in col]
    tfidf_cols = [col for col in df.columns if 'TFIDF' in col]
    other_cols = ['MARKETING_YEAR']

    # Combine all feature columns
    all_feature_cols = text_cols + length_cols + count_cols + binary_cols + tfidf_cols + other_cols

    # Filter to only include columns that actually exist in the dataframe
    feature_cols = [col for col in all_feature_cols if col in df.columns]

    # 5.2 Define features and target
    y = df['GROUPED_MARKETING_CATEGORY']
    X = df[feature_cols]

    # 5.3 Split data with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y)

    # Log class distribution
    print("Class distribution in training set:")
    print(y_train.value_counts(normalize=True))

    print("Class distribution in test set:")
    print(y_test.value_counts(normalize=True))

    # Free up memory
    del X, y
    gc.collect()

    print(f"Train set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
    print(f"Test set: {X_test.shape[0]} samples, {X_test.shape[1]} features")
    return X_train, X_test, y_train, y_test, feature_cols

# ML Model Traning - Random Forest, Logistic Regression and XG-Boost

Logistic Regression is run to show linear model is not suitable for this methods



In [7]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, feature_cols):
    """
    Train and evaluate multiple machine learning models
    """
    print("Starting model training and evaluation...")

    # Encode target variable if it's categorical
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Store the mapping for later reference
    class_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
    print("Class mapping:", class_mapping)

    # Save the label encoder for future use
    joblib.dump(label_encoder, 'models/label_encoder.pkl')

    # Create pipelines for text and numeric features
    # For text features, we'll use a separate pipeline
    text_cols = ['PROPRIETARYNAME', 'DOSAGEFORMNAME', 'ROUTENAME', 'LABELERNAME']
    text_cols = [col for col in text_cols if col in X_train.columns]

    # For numeric features, we'll use a simple imputer
    numeric_cols = [col for col in X_train.columns if col not in text_cols]

    # Random Forest Model
    print("\nTraining Random Forest Classifier...")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1  # Use all available cores
    )

    # Encode all text columns before training
    text_cols = ['PROPRIETARYNAME', 'DOSAGEFORMNAME', 'ROUTENAME', 'LABELERNAME']
    text_cols = [col for col in text_cols if col in X_train.columns]

    for col in text_cols:
      le = LabelEncoder()
      all_values = pd.concat([X_train[col], X_test[col]]).astype(str)
      le.fit(all_values)

      X_train[col] = le.transform(X_train[col].astype(str))
      X_test[col] = le.transform(X_test[col].astype(str))

      # Optional: Save the encoders if needed later
      joblib.dump(le, f'models/{col}_encoder_rf.pkl')


    # Train the model
    rf_model.fit(X_train, y_train)

    # Make predictions
    rf_pred = rf_model.predict(X_test)
    rf_pred_proba = rf_model.predict_proba(X_test)

    # Evaluate the model
    print("\nRandom Forest Classification Report:")
    print(classification_report(y_test, rf_pred))

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_test, rf_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Random Forest Confusion Matrix')
    plt.tight_layout()
    plt.savefig('outputs/rf_confusion_matrix.jpg')
    plt.close()

    # Feature importance
    plt.figure(figsize=(12, 10))
    importances = rf_model.feature_importances_
    indices = np.argsort(importances)[::-1]
    plt.title('Random Forest Feature Importances')
    plt.bar(range(min(20, len(importances))),
            importances[indices][:20],
            color='royalblue',
            align='center')
    plt.xticks(range(min(20, len(importances))),
               np.array(X_train.columns)[indices][:20],
               rotation=90)
    plt.tight_layout()
    plt.savefig('outputs/rf_feature_importance.jpg')
    plt.close()

    # Plot ROC curves
    plt.figure(figsize=(12, 10))
    n_classes = len(label_encoder.classes_)

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(
            (y_test_encoded == i).astype(int),
            rf_pred_proba[:, i]
        )
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2,
                 label=f'ROC curve for {class_mapping[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Random Forest: ROC Curves per Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('outputs/random_forest_roc_curves.jpg')
    plt.close()

    # Save the model
    joblib.dump(rf_model, 'models/random_forest_model.pkl')
    print("Random Forest model saved to models/random_forest_model.pkl")

    # Release memory
    del rf_model, rf_pred, rf_pred_proba
    gc.collect()

    imputer = SimpleImputer(strategy='mean')  # or 'most_frequent' for categorical

    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

    # Logistic Regression Model
    print("\nTraining Logistic Regression Classifier...")
    lr_model = LogisticRegression(
        C=1.0,
        max_iter=1000,
        multi_class='multinomial',
        solver='saga',
        n_jobs=-1,
        random_state=42
    )

    # Train the model
    #lr_model.fit(X_train, y_train)
    lr_model.fit(X_train_imputed, y_train)

    # Making predictions
    # lr_pred = lr_model.predict(X_test)
    # lr_pred_proba = lr_model.predict_proba(X_test)
    lr_pred = lr_model.predict(X_test_imputed)
    lr_pred_proba = lr_model.predict_proba(X_test_imputed)


    # Evaluate the model
    print("\nLogistic Regression Classification Report:")
    print(classification_report(y_test, lr_pred))

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_test, lr_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Logistic Regression Confusion Matrix')
    plt.tight_layout()
    plt.savefig('outputs/lr_confusion_matrix.jpg')
    plt.close()

    # Plot ROC curves
    plt.figure(figsize=(12, 10))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(
            (y_test_encoded == i).astype(int),
            lr_pred_proba[:, i]
        )
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2,
                 label=f'ROC curve for {class_mapping[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Logistic Regression: ROC Curves per Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('outputs/logistic_regression_roc_curves.jpg')
    plt.close()

    # Save the model
    joblib.dump(lr_model, 'models/logistic_regression_model.pkl')
    print("Logistic Regression model saved to models/logistic_regression_model.pkl")

    # Release memory
    del lr_model, lr_pred, lr_pred_proba
    gc.collect()

    # XGBoost Model
    print("\nTraining XGBoost Classifier...")
    xgb_model = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        use_label_encoder=False,
        eval_metric='mlogloss'
    )

    # Train the model
    xgb_model.fit(X_train, y_train_encoded)

    # Make predictions
    xgb_pred = xgb_model.predict(X_test)
    xgb_pred_class = label_encoder.inverse_transform(xgb_pred)
    xgb_pred_proba = xgb_model.predict_proba(X_test)

    # Evaluate the model
    print("\nXGBoost Classification Report:")
    print(classification_report(y_test, xgb_pred_class))

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_test, xgb_pred_class)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('XGBoost Confusion Matrix')
    plt.tight_layout()
    plt.savefig('outputs/xgb_confusion_matrix.jpg')
    plt.close()

    # Feature importance
    plt.figure(figsize=(12, 10))
    xgb_importance = xgb_model.feature_importances_
    indices = np.argsort(xgb_importance)[::-1]
    plt.title('XGBoost Feature Importances')
    plt.bar(range(min(20, len(xgb_importance))),
            xgb_importance[indices][:20],
            color='forestgreen',
            align='center')
    plt.xticks(range(min(20, len(xgb_importance))),
               np.array(X_train.columns)[indices][:20],
               rotation=90)
    plt.tight_layout()
    plt.savefig('outputs/xgb_feature_importance.jpg')
    plt.close()

    # Plot ROC curves
    plt.figure(figsize=(12, 10))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(
            (y_test_encoded == i).astype(int),
            xgb_pred_proba[:, i]
        )
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2,
                 label=f'ROC curve for {class_mapping[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('XGBoost: ROC Curves per Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('outputs/xgboost_roc_curves.jpg')
    plt.close()

    # Save the model
    joblib.dump(xgb_model, 'models/xgboost_model.pkl')
    print("XGBoost model saved to models/xgboost_model.pkl")

    # Exporting test predictions for all models to csv
    print("\nExporting predictions to CSV...")

    # Reload models to ensure we're using the saved versions
    rf_model = joblib.load('models/random_forest_model.pkl')
    lr_model = joblib.load('models/logistic_regression_model.pkl')
    xgb_model = joblib.load('models/xgboost_model.pkl')

    # Generate predictions
    rf_preds = rf_model.predict(X_test)
    # lr_preds = lr_model.predict(X_test)
    lr_preds = lr_model.predict(X_test_imputed)
    xgb_preds = label_encoder.inverse_transform(xgb_model.predict(X_test))

    # Create a DataFrame with predictions
    predictions_df = pd.DataFrame({
        'true_label': y_test.values,
        'rf_prediction': rf_preds,
        'lr_prediction': lr_preds,
        'xgb_prediction': xgb_preds
    })

    # Save to CSV
    predictions_df.to_csv('outputs/test_predictions.csv', index=False)
    print("Predictions saved to outputs/test_predictions.csv")

    # SHAP values for model interpretability (for Random Forest)
    print("\nCalculating SHAP values for model interpretability...")
    try:
        # Create a smaller sample for SHAP analysis to avoid memory issues
        sample_size = min(1000, X_test.shape[0])
        X_sample = X_test.sample(sample_size, random_state=42)

        # Calculate SHAP values for Random Forest
        explainer = shap.TreeExplainer(rf_model)
        shap_values = explainer.shap_values(X_sample)

        # Plot summary
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
        plt.title('SHAP Feature Importance')
        plt.tight_layout()
        plt.savefig('outputs/shap_feature_importance.jpg')
        plt.close()

        # Plot detailed SHAP values for top class
        plt.figure(figsize=(12, 10))
        shap.summary_plot(shap_values[0], X_sample, show=False)
        plt.title(f'SHAP Values for Class: {label_encoder.classes_[0]}')
        plt.tight_layout()
        plt.savefig('outputs/shap_values_detail.jpg')
        plt.close()

        print("SHAP analysis complete and visualizations saved.")
    except Exception as e:
        print(f"SHAP analysis error: {e}")
        print("Skipping SHAP analysis due to error.")

    # Release memory
    del rf_model, lr_model, xgb_model
    gc.collect()

    return

# Deep Leanring Model - Multi Layer Perceptron(MLP) Neural Network

In [8]:
def train_deep_learning_model(X_train, X_test, y_train, y_test):
    """
    Train and evaluate a deep learning model
    """
    print("\nTraining Deep Learning Model...")

    # Check if GPU is available
    if tf.config.list_physical_devices('GPU'):
        print("GPU is available. Using GPU for training.")
    else:
        print("GPU is not available. Using CPU for training.")

    # Encode target variable if it's categorical
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Convert to one-hot encoding
    n_classes = len(label_encoder.classes_)
    y_train_onehot = to_categorical(y_train_encoded, num_classes=n_classes)
    y_test_onehot = to_categorical(y_test_encoded, num_classes=n_classes)

    # Print class mapping
    class_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
    print("Class mapping:", class_mapping)

    # Separate features by type
    # Text features
    text_cols = ['PROPRIETARYNAME', 'DOSAGEFORMNAME', 'ROUTENAME', 'LABELERNAME']
    text_cols = [col for col in text_cols if col in X_train.columns]

    # Numeric features
    numeric_cols = [col for col in X_train.columns if col not in text_cols]

    # Fill NaNs in numeric features
    X_train[numeric_cols] = X_train[numeric_cols].fillna(0)
    X_test[numeric_cols] = X_test[numeric_cols].fillna(0)

    # Convert to numpy arrays
    X_train_numeric = X_train[numeric_cols].values
    X_test_numeric = X_test[numeric_cols].values

    # Normalize numeric features
    numeric_mean = X_train_numeric.mean(axis=0)
    numeric_std = X_train_numeric.std(axis=0)
    numeric_std[numeric_std == 0] = 1  # Avoid division by zero
    X_train_numeric = (X_train_numeric - numeric_mean) / numeric_std
    X_test_numeric = (X_test_numeric - numeric_mean) / numeric_std

    # Save normalizer parameters
    np.save('models/numeric_mean.npy', numeric_mean)
    np.save('models/numeric_std.npy', numeric_std)

    # Building model
    print("Building neural network model...")

    # Input layer for numeric features
    numeric_input = Input(shape=(X_train_numeric.shape[1],), name='numeric_input')

    # First hidden layer with batch normalization
    x = Dense(128, activation='relu')(numeric_input)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Second hidden layer
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # Output layer
    output = Dense(n_classes, activation='softmax')(x)

    # Compile model
    model = Model(inputs=numeric_input, outputs=output)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Print model summary
    model.summary()

    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    # Train model
    print("\nTraining neural network...")
    history = model.fit(
        X_train_numeric,
        y_train_onehot,
        epochs=20,
        batch_size=256,
        validation_split=0.2,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Evaluate model
    print("\nEvaluating neural network...")
    test_loss, test_acc = model.evaluate(X_test_numeric, y_test_onehot, verbose=0)
    print(f"Test accuracy: {test_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}")

    # Making predictions
    dl_pred_proba = model.predict(X_test_numeric)
    dl_pred_class = np.argmax(dl_pred_proba, axis=1)
    dl_pred = label_encoder.inverse_transform(dl_pred_class)

    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.tight_layout()
    plt.savefig('outputs/dl_training_history.jpg')
    plt.close()

    # Print classification report
    print("\nDeep Learning Classification Report:")
    print(classification_report(y_test, dl_pred))

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_test, dl_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Deep Learning Model Confusion Matrix')
    plt.tight_layout()
    plt.savefig('outputs/dl_confusion_matrix.jpg')
    plt.close()

    # Plot ROC curves
    plt.figure(figsize=(12, 10))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(
            (y_test_encoded == i).astype(int),
            dl_pred_proba[:, i]
        )
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2,
                 label=f'ROC curve for {class_mapping[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Deep Learning: ROC Curves per Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('outputs/deep_learning_roc_curves.jpg')
    plt.close()

    # Saving the model
    model.save('models/dl_model.keras')
    print("Deep Learning model saved to models/dl_model.keras")

    # Release memory
    del model, history
    gc.collect()

    return

# Improving Deep Learning Model with Embeddings

In [9]:
def train_deep_learning_model_with_embeddings(X_train, X_test, y_train, y_test):
    """
    Train and evaluate an improved deep learning model with embeddings for categorical features
    """
    print("\nTraining Improved Deep Learning Model with Embeddings...")

    # Checking if GPU is available
    if tf.config.list_physical_devices('GPU'):
        print("GPU is available. Using GPU for training.")
    else:
        print("GPU is not available. Using CPU for training.")

    # Encode target variable if it's categorical
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Convert to one-hot encoding
    n_classes = len(label_encoder.classes_)
    y_train_onehot = to_categorical(y_train_encoded, num_classes=n_classes)
    y_test_onehot = to_categorical(y_test_encoded, num_classes=n_classes)

    # Print class mapping
    class_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
    print("Class mapping:", class_mapping)

    # Separating features by type
    # Text features to use for embeddings
    text_cols = ['PROPRIETARYNAME', 'DOSAGEFORMNAME', 'ROUTENAME', 'LABELERNAME']
    text_cols = [col for col in text_cols if col in X_train.columns]

    # Numeric features
    numeric_cols = [col for col in X_train.columns if col not in text_cols]

    X_train[numeric_cols] = X_train[numeric_cols].fillna(0)
    X_test[numeric_cols] = X_test[numeric_cols].fillna(0)

    # Convert numeric data to numpy arrays
    X_train_numeric = X_train[numeric_cols].values
    X_test_numeric = X_test[numeric_cols].values

    # Normalize numeric features
    numeric_mean = X_train_numeric.mean(axis=0)
    numeric_std = X_train_numeric.std(axis=0)
    numeric_std[numeric_std == 0] = 1  # Avoid division by zero
    X_train_numeric = (X_train_numeric - numeric_mean) / numeric_std
    X_test_numeric = (X_test_numeric - numeric_mean) / numeric_std

    # Save normalizer parameters
    np.save('models/dl_improved_numeric_mean.npy', numeric_mean)
    np.save('models/dl_improved_numeric_std.npy', numeric_std)

    # Preparing text data for embeddings
    print("Processing text features for embeddings...")
    # Dictionary to store label encoders for each text column
    text_encoders = {}

    # Process each text column
    for col in text_cols:
        print(f"Encoding text column: {col}")
        # Create and fit a label encoder for this column
        col_encoder = LabelEncoder()
        # Combine train and test values to ensure all categories are represented
        all_values = pd.concat([X_train[col], X_test[col]]).unique()
        col_encoder.fit(all_values)

        # Transform train and test data
        X_train[f"{col}_encoded"] = col_encoder.transform(X_train[col])
        X_test[f"{col}_encoded"] = col_encoder.transform(X_test[col])

        # Store the encoder
        text_encoders[col] = col_encoder

        # Save the encoder
        joblib.dump(col_encoder, f'models/{col}_encoder.pkl')
        print(f"  - {col} has {len(col_encoder.classes_)} unique values")

    # Building model with embeddings
    print("Building neural network model with embeddings...")

    # Input layer for numeric features
    numeric_input = Input(shape=(X_train_numeric.shape[1],), name='numeric_input')
    numeric_dense = Dense(64, activation='relu')(numeric_input)
    numeric_bn = BatchNormalization()(numeric_dense)

    # Input layers and embedding layers for text features
    text_inputs = []
    text_embeddings = []

    for col in text_cols:
        # Get vocabulary size
        vocab_size = len(text_encoders[col].classes_)

        # Create input layer
        text_input = Input(shape=(1,), name=f'{col}_input')
        text_inputs.append(text_input)

        # Create embedding layer (embedding size depends on vocabulary size)
        embedding_size = min(50, (vocab_size + 1) // 2)  # Rule of thumb
        embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size,
                             name=f'{col}_embedding')(text_input)
        embedding = Flatten(name=f'{col}_flatten')(embedding)
        text_embeddings.append(embedding)

    # Concatenate all inputs
    if text_embeddings:
        concatenated = Concatenate()([numeric_bn] + text_embeddings)
    else:
        concatenated = numeric_bn

    # Hidden layers
    x = Dense(128, activation='relu')(concatenated)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # Output layer
    output = Dense(n_classes, activation='softmax')(x)

    # Compile model
    if text_inputs:
        model = Model(inputs=[numeric_input] + text_inputs, outputs=output)
    else:
        model = Model(inputs=numeric_input, outputs=output)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Print model summary
    model.summary()

    # Define callbacks to stop the model by overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )

    # Prepare inputs for training
    train_inputs = [X_train_numeric]
    test_inputs = [X_test_numeric]

    for col in text_cols:
        train_inputs.append(X_train[f"{col}_encoded"].values.reshape(-1, 1))
        test_inputs.append(X_test[f"{col}_encoded"].values.reshape(-1, 1))

    # Train model
    print("\nTraining improved neural network with embeddings...")
    history = model.fit(
        train_inputs,
        y_train_onehot,
        epochs=30,
        batch_size=256,
        validation_split=0.2,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Evaluate model
    print("\nEvaluating improved neural network...")
    test_loss, test_acc = model.evaluate(test_inputs, y_test_onehot, verbose=0)
    print(f"Test accuracy: {test_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}")

    # Make predictions
    dl_pred_proba = model.predict(test_inputs)
    dl_pred_class = np.argmax(dl_pred_proba, axis=1)
    dl_pred = label_encoder.inverse_transform(dl_pred_class)

    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.tight_layout()
    plt.savefig('outputs/dl_improved_training_history.jpg')
    plt.close()

    # Print classification report
    print("\nImproved Deep Learning Classification Report:")
    print(classification_report(y_test, dl_pred))

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_test, dl_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
              xticklabels=label_encoder.classes_,
              yticklabels=label_encoder.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Improved Deep Learning Model Confusion Matrix')
    plt.tight_layout()
    plt.savefig('outputs/dl_improved_confusion_matrix.jpg')
    plt.close()

    # Plot ROC curves
    plt.figure(figsize=(12, 10))
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(
            (y_test_encoded == i).astype(int),
            dl_pred_proba[:, i]
        )
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2,
               label=f'ROC curve for {class_mapping[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Improved Deep Learning: ROC Curves per Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('outputs/deep_learning_improved_roc_curves.jpg')
    plt.close()

    # Save the model
    model.save('models/dl_improved_model.keras')
    print("Improved Deep Learning model saved to models/dl_improved_model.keras")

    # Release memory
    del model, history
    gc.collect()
    return

# Execution Fuction

In [10]:
def main(data_path):
    """
    Main function to orchestrate the entire pipeline
    """
    print("Starting FDA Drug Product Classification pipeline...")

    # Load data
    df = load_data(data_path)
    if df is None:
        print("Failed to load data. Exiting.")
        return

    # Clean and preprocess data
    df = clean_and_preprocess_data(df)

    # Engineer features
    df = engineer_features(df)

    # Prepare data for modeling
    X_train, X_test, y_train, y_test, feature_cols = prepare_data_for_modeling(df)

    # Train and evaluate machine learning models
    train_and_evaluate_models(X_train, X_test, y_train, y_test, feature_cols)

    # Train and evaluate basic deep learning model
    train_deep_learning_model(X_train, X_test, y_train, y_test)

    # Train and evaluate improved deep learning model with embeddings
    train_deep_learning_model_with_embeddings(X_train, X_test, y_train, y_test)

    # Print final summary
    print("\n" + "="*50)
    print("FDA Drug Product Classification Pipeline Complete")
    print("="*50)
    print("\nOutput files are saved in the 'outputs' directory.")
    print("Trained models are saved in the 'models' directory.")
    print("\nSummary of models:")
    print("1. Random Forest")
    print("2. Logistic Regression")
    print("3. XGBoost")
    print("4. Basic Deep Learning Model")
    print("5. Improved Deep Learning Model with Embeddings")
    print("\nSee classification reports and visualizations for performance metrics.")

    return

In [12]:
if __name__ == "__main__":
    # Specify the path to your data
    # data_path = "/content/drive/MyDrive/ML_Project/fda_product.xlsx"
    data_path = "/content/fda_product.xlsx"

    # Run the main function
    main(data_path)

Starting FDA Drug Product Classification pipeline...
Loading data from /content/fda_product.xlsx...
Data loaded successfully with 111970 rows and 9 columns.
Memory usage: 7.69 MB
Starting data cleaning and preprocessing...
Data cleaned and preprocessed. Remaining rows: 111962
Starting feature engineering...
Generating TF-IDF features for SUBSTANCENAME...
Top TF-IDF terms:
        acetaminophen
        acetate
        acid
        alcohol
        aluminum
        arsenic
        avobenzone
        bark
        benzalkonium
        calcium
Feature engineering completed
Memory usage after engineering: 22.21 MB
Preparing data for modeling...
Class distribution in training set:
GROUPED_MARKETING_CATEGORY
ANDA          0.434883
OTC           0.302080
UNAPPROVED    0.146948
NDA           0.079737
BLA           0.036352
Name: proportion, dtype: float64
Class distribution in test set:
GROUPED_MARKETING_CATEGORY
ANDA          0.434868
OTC           0.302103
UNAPPROVED    0.146966
NDA           0

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ numeric_input (InputLayer)      │ (None, 78)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        10,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,461 (76.02 KB)

 Trainable params: 19,077 (74.52 KB)

...

 Total params: 2,336,781 (8.91 MB)

 Trainable params: 2,336,269 (8.91 MB)

 Non-trainable params: 512 (2.00 KB)


Training improved neural network with embeddings...
Epoch 1/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.7343 - loss: 0.7838 - val_accuracy: 0.9387 - val_loss: 0.1896 - learning_rate: 0.0010
Epoch 2/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.9474 - loss: 0.1579 - val_accuracy: 0.9592 - val_loss: 0.1226 - learning_rate: 0.0010
Epoch 3/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.9769 - loss: 0.0734 - val_accuracy: 0.9614 - val_loss: 0.1206 - learning_rate: 0.0010
Epoch 4/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.9844 - loss: 0.0488 - val_accuracy: 0.9624 - val_loss: 0.1219 - learning_rate: 0.0010
Epoch 5/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.9871 - loss: 0.0411 - val_accuracy: 0.9622 - val_loss: 0.1231 - learning_rate: 0.0010
Epoch 6/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.9880 - loss: 0.0355 - val_accuracy: 0.9613 - val_loss: 0.1286 - learning_rate: 0.0010
Epoch 7/30
280/280 ━━━━━━━

<Figure size 1200x1000 with 0 Axes>